In [ ]:
# !pip install openai
# !pip install tqdm

In [ ]:
!python --version

In [ ]:
import json
import random

import numpy as np
import openai
import pandas as pd
from tqdm.notebook import tqdm

openai.api_key = "sk-WYfx2w0AwDkwBxyixWfl77Cy2VYhOgWvQOlQbVqc"

In [ ]:
STATUS_CHECK = "status_check"
FEES_INQUIRY = "fees_inquiry"
GET_CREDENTIALS = "get_credentials"
CHANGE_CREDENTIALS = "change_credential_info"
ACCOUNT_ACTIVITY = "get_account_activity"
ACCOUNT_BALANCE = "get_account_balance"
# -------------------------------------------
DUES_CHECK = "dues"
ACTIVATE = "activate"
REQUEST_CARD_ACTION = "replace_cancel_report_missing_card"
REQUEST_INFO_INTENT = "ask_for_info"
APPLY_FOR = "apply_for"

In [ ]:
card_brands = [
    "Smiles",
    "Emirates",
    "Cashback",
    "Skywards",
    "Rewards",
    "Etihad",
    "Skyward",
    "Spice",
    "amex",
    "diners",
    "discover",
    "jcb",
    "jcb15",
    "jcb16",
    "maestro",
    "mastercard",
    "visa",
    "visa13",
    "visa16",
    "visa19",
]
account_types = [
    "salary",
    "ADIB",
    "Ghina",
    "recurring deposit",
    "fixed deposit",
    "demat",
    "",
]
debt_info_terms = [
    "amount due",
    "due amount",
    "payment date",
    "installment",
]
debt_types = ["car", "travel", "personal", "student", "education", "housing", "home"]
debt_types_with_blanks = debt_types + [""] * len(debt_types)
debt_synonyms = ["financing", "debt", "loan", "loans", "finance"]

In [ ]:
c1 = np.random.choice(card_brands)
c2 = np.random.choice(card_brands)
debt_syn = np.random.choice(debt_synonyms)

prompts = {
    "transfer": """
Keywords: bank account, transfer, savings
Text: How to transfer money from my savings account to my friends account?

Keywords: bank account, transfer, savings
Text: I want money in my current account from savings. How do I do that?

Keywords: bank account, transfer, ADIB
Text: What is the process to transfer money from or to my ADIB account?
""",
    "fees": """
Keywords: bank account, fees, savings
Text: What is the fees for this savings account?
""",
    "status_check": """
Keywords: bank account, status check, Ghina
Text: What is the status of my Ghina account application?
# """,
    #     DUES_CHECK: f"""
    # Keywords: {DUES_CHECK}, credit card, Amex
    # Text: What is the pending amount on my Amex?
    # Keywords: {DUES_CHECK}, credit card, {c1}
    # Text: When is the last date to pay my dues on the {c1}?
    # Keywords: {DUES_CHECK}, credit card, {c2}
    # Text: What is this month's instalment amount on my {c2.upper()} Credit Card?
    # Keywords: {DUES_CHECK}, credit card, {c2}
    # Text: How can I clear by {c2} dues?
    #     """,
    DUES_CHECK: f"""
Keywords: {DUES_CHECK}, loan, personal
Text: What is the pending amount of my personal loan?

Keywords: {DUES_CHECK}, loan, home
Text: f"How much remaining in my home {debt_syn}"

Keywords: {DUES_CHECK}, debt
Text: When is the last date to pay my dues?

Keywords: {DUES_CHECK}, financing
Text: What is this month's instalment amount on my financing?

Keywords: {DUES_CHECK}, debt
Text: How to make payment for instalment? What payment modes?
""",
    ACTIVATE: f"""
Keywords: {ACTIVATE}, credit card
Text: Pleae activate my credit card

Keywords: {ACTIVATE}, online banking, bank account
Text: How to turn on online banking on my account?

Keywords: {ACTIVATE}, online banking
Text: I want to know the process to activate online account access
    """,
    REQUEST_CARD_ACTION: f"""
Keywords: Report Missing, Credit Card, VISA
Text: report my VISA credit card as stolen. please block it

Keywords: Cancel, Etihad Card
Text: Please cancel my Etihad card as I no longer use it

Keywords: Replace, Mastercard
Text: Request to please replace my blocked Mastercard card
    """,
}

In [ ]:
# from dataclasses import dataclass
# @dataclass
# class Config:
#     max_tokens: int
#     temperature: float
#     presence_penalty: float
#     frequency_penalty: float

# c = Config(300, 0.8, 0.4, 0.4)

In [ ]:
prompts.keys()

In [ ]:
np.random.seed(39)
intent = REQUEST_CARD_ACTION
engine = "davinci"
max_tokens_large = 500
max_tokens_usual = 300
configurations = [
    {
        "engine": engine,
        "prompt": f"{prompts[intent]}\nKeywords: Replace, Etihad card\nText:",
        "max_tokens": max_tokens_large,
        "temperature": 0.4,
        "presence_penalty": 0.4,
        "frequency_penalty": 0.9,
    },
    {
        "engine": engine,
        "prompt": f"{prompts[intent]}\nKeywords: Return, Diners card \nText:",
        "max_tokens": max_tokens_large,
        "temperature": 0.75,
        "presence_penalty": 0.7,
        "frequency_penalty": 0.6,
    },
    {
        "engine": engine,
        "prompt": f"{prompts[intent]}\nKeywords: Close, Amex Credit card \nText:",
        "max_tokens": max_tokens_large,
        "temperature": 0.6,
        "presence_penalty": 0.4,
        "frequency_penalty": 0.5,
    },
    {
        "engine": engine,
        "prompt": f"{prompts[intent]}\nKeywords: Block, VISA, Credit card \nText:",
        "max_tokens": max_tokens_usual,
        "temperature": 0.6,
#         "presence_penalty": 0.4,
        "frequency_penalty": 0.7,
    },
    {
        "engine": engine,
        "prompt": f"{prompts[intent]}\nKeywords: Report Stolen, Mastercard \nText:",
        "max_tokens": max_tokens_usual,
        "temperature": 0.6,
#         "presence_penalty": 0.4,
        "frequency_penalty": 0.5,
    },
]
# configurations.extend(
#     [
#         {
#             "engine": engine,
#             "prompt": f"{prompts[intent]}\nKeywords: {intent}, atm, OTP \nText:",
#             "max_tokens": 300,
#             "temperature": np.round(x, 2),
#             "frequency_penalty": np.round(np.random.uniform(0.3, 1.0), 2),
#         }
#         for x in np.arange(0.4, 0.7, 0.05)
#     ]
# )
configurations *= 3
configurations

In [ ]:
def response_parser(openai_response: object):
    """
    response: OpenAI object response
    """
    response = json.loads(str(openai_response))["choices"][0][
        "text"
    ]  
    keyword = "Keywords: "
    text = "Text: "
    
    parsed_response = []
    for entries in response.strip().split("\n\n")[1:]:
        entry = entries.split("\n")
        keywords_string = entry[0].strip().replace(keyword, "")
        parsed_entry = {
            "keywords": keywords_string.split(","),
            "sentence": entry[1].strip().replace(text, "").replace("Text:", ""),
        }
        parsed_response.append(parsed_entry)
    return parsed_response


parsed_responses, unparsed_responses = [], []
for config in tqdm(configurations):
    response = openai.Completion.create(
    **config,
    )
    try:
        parsed = response_parser(response)
    except:
        unparsed_responses.append(response)
    else:
#         print(parsed)
        parsed_responses.extend(parsed)
    

In [ ]:
len(parsed_responses), len(unparsed_responses)

In [ ]:
parsed_responses, unparsed_responses

In [ ]:
with open("../data/test_drive_30.json", "w") as f:
    json.dump(parsed_responses, f, indent=2)